In [1]:
import pandas as pd
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample

In [2]:


#dosyayı oku
df = pd.read_excel("C:\\Users\\Sariye\\Desktop\\YZM-3\\NLP\\IMDBtop25_yorumlari(temizlenmis_hali).xlsx")  # Dosya yolunu doğru şekilde değiştirin

# NLTK kütüphanesinden Vader duygusal analiz sözlüğünü indirin
nltk.download('vader_lexicon')

# SentimentIntensityAnalyzer sınıfını oluşturun
sentiments = SentimentIntensityAnalyzer()

# Duygusal skorları hesaplayarak veri çerçevesine ekleyin
df["Positive"] = [sentiments.polarity_scores(comment)["pos"] for comment in df["body"]]
df["Negative"] = [sentiments.polarity_scores(comment)["neg"] for comment in df["body"]]
df["Neutral"] = [sentiments.polarity_scores(comment)["neu"] for comment in df["body"]]
df['Compound'] = [sentiments.polarity_scores(comment)["compound"] for comment in df["body"]]

# Duygusal skorlara dayalı olarak etiketleme yapın
score = df["Compound"].values
sentiment = []

for i in score:
    if i >= 0.05 :
        sentiment.append('Positive')
    elif i <= -0.05 :
        sentiment.append('Negative')
    else:
        sentiment.append('Neutral')
df["Sentiment"] = sentiment
df.head()


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Sariye\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,body,Positive,Negative,Neutral,Compound,Sentiment
0,shawshank redemption written directed frank da...,0.000,0.000,1.000,0.0000,Neutral
1,adaptation stephen king novella rita hayworth ...,0.000,0.000,1.000,0.0000,Neutral
2,starring tim robbins morgan freeman film portr...,0.231,0.260,0.509,-0.2500,Negative
3,andy finds tough going finds solace friendship...,0.309,0.083,0.608,0.6249,Positive
4,things start pick warden finds andy prison job...,0.196,0.216,0.588,-0.0772,Negative


In [3]:
pd.options.display.max_rows = None

In [4]:

data2=df.drop(['Positive','Negative','Neutral','Compound'],axis=1)

In [5]:
data2.to_excel("IMDBtop25_yorumlari(labeling)1.xlsx" , index=False, engine='openpyxl')

In [6]:
nltk.download('omw-1.4')
data_copy = data2.copy()

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Sariye\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [7]:
le = LabelEncoder()
data_copy['Sentiment'] = le.fit_transform(data_copy['Sentiment'])

In [8]:
processed_data = {
    'Sentence':data_copy.body,
    'Sentiment':data_copy['Sentiment']
}

processed_data = pd.DataFrame(processed_data)
processed_data.head()

,Sentence,Sentiment
0,shawshank redemption written directed frank da...,1
1,adaptation stephen king novella rita hayworth ...,1
2,starring tim robbins morgan freeman film portr...,0
3,andy finds tough going finds solace friendship...,2
4,things start pick warden finds andy prison job...,0


In [9]:
processed_data['Sentiment'].value_counts()

Sentiment
2    5774
1    2610
0    2508
Name: count, dtype: int64

In [10]:
processed_data['Sentiment'].value_counts()

Sentiment
2    5774
1    2610
0    2508
Name: count, dtype: int64

In [11]:
corpus = []
for sentence in processed_data['Sentence']:
    corpus.append(sentence)

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=1500)
X = cv.fit_transform(corpus).toarray()
y = processed_data.iloc[:, -1].values

In [13]:
#model eğitimi
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
classifier = GaussianNB()
classifier.fit(X_train, y_train)

GaussianNB()

In [14]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
cm

array([[386, 334,  67],
       [ 57, 700,  24],
       [435, 661, 604]], dtype=int64)

In [15]:
nb_score = accuracy_score(y_test, y_pred)
print('accuracy',nb_score)

accuracy 0.5171358629130967


In [16]:
# Örnek bir yorumu tahmin etme
example_comment = ["This is awesome"]
example_comment_transformed = cv.transform(example_comment).toarray()
prediction = classifier.predict(example_comment_transformed)

# Tahmin sonucunu yazdırma
print('Tahmin Sonucu:', le.inverse_transform(prediction))

Tahmin Sonucu: ['Positive']


In [17]:
# Örnek bir yorumu tahmin etme
example_comment = ["classic film"]
example_comment_transformed = cv.transform(example_comment).toarray()
prediction = classifier.predict(example_comment_transformed)

# Tahmin sonucunu yazdırma
print('Tahmin Sonucu:', le.inverse_transform(prediction))

Tahmin Sonucu: ['Neutral']
